In [ ]:
# %% Minimal setup
# If needed (uncomment in a notebook):
# !pip install requests python-dotenv

import os, json, textwrap, re, time
import requests

API_KEY  = os.getenv("OPENAI_API_KEY", "cse476")
API_BASE = os.getenv("API_BASE", "http://10.4.58.53:41701/v1")  
MODEL    = os.getenv("MODEL_NAME", "bens_model")              

def call_model_chat_completions(prompt: str,
                                system: str = "You are a helpful assistant. Reply with only the final answer—no explanation.",
                                model: str = MODEL,
                                temperature: float = 0.3,
                                timeout: int = 60,
                                max_tokens: int = 128) -> dict:
    """
    Calls an OpenAI-style /v1/chat/completions endpoint and returns:
    { 'ok': bool, 'text': str or None, 'raw': dict or None, 'status': int, 'error': str or None, 'headers': dict }
    """
    url = f"{API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json",
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user",   "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
    }

    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        status = resp.status_code
        hdrs   = dict(resp.headers)
        if status == 200:
            data = resp.json()
            text = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            return {"ok": True, "text": text, "raw": data, "status": status, "error": None, "headers": hdrs}
        else:
            # try best-effort to surface error text
            err_text = None
            try:
                err_text = resp.json()
            except Exception:
                err_text = resp.text
            return {"ok": False, "text": None, "raw": None, "status": status, "error": str(err_text), "headers": hdrs}
    except requests.RequestException as e:
        return {"ok": False, "text": None, "raw": None, "status": -1, "error": str(e), "headers": {}}


In [20]:
# %% Direct call example
def direct_call(prompt="What is 17 + 28? Answer with just the number.", temperature=0.2, max_tokens=128):
    demo_prompt = prompt
    result = call_model_chat_completions(demo_prompt, temperature=temperature, max_tokens=max_tokens)
    print("OK:", result["ok"], "HTTP:", result["status"])
    print("MODEL SAYS:", (result["text"] or "").strip())

    # Optional: Inspect rate-limit headers if your provider exposes them
    for k in ["x-ratelimit-remaining-requests", "x-ratelimit-limit-requests", "x-request-id"]:
        if k in result["headers"]:
            print(f"{k}: {result['headers'][k]}")


In [21]:
# %% Define three tests: input + expected
my_tests = [
    {
        "id": "math_inequality",
        "type": "numeric",  # grader will prefer numeric extraction
        "prompt": "Solve for the smallest integer n such that 3n + 5 > 26. Answer with just the integer.",
        "expected": "8",    # Because 3n > 21 => n > 7, smallest integer is 8
    },
    {
        "id": "commonsense_ice",
        "type": "text",
        "prompt": (
            "You place an ice cube in a glass of water and mark the water level. "
            "After the ice melts, does the water level rise, fall, or stay the same? "
            "Answer with exactly one of: 'rise', 'fall', 'stay the same'."
        ),
        "expected": "stay the same",
    },
    {
        "id": "logic_race",
        "type": "text",
        "prompt": (
            "In a race, you pass the person in second place. What position are you now in? "
            "Answer with a single word like 'first', 'second', 'third'."
        ),
        "expected": "second",
    },
]


In [61]:
import json
from pprint import pprint
import random

POSSIBLE_TYPES = ['math', 'common_sense', 'planning', 'coding', 'future_prediction']

all_tests = json.load(open("parsed_dev_data.json", "r", encoding="utf-8"))

formatted_tests = []
for i, t in enumerate(all_tests, start=1):
    
    formatted_tests.append({
        "id": t['id'], # domain_domainIndex_domainTestIndex_testIndex
        "type": t['domain'],
        "prompt": t['input'],
        "expected": t['output'],
        "char_count": t['input_char_count']
    })
    
all_tests = formatted_tests

In [ ]:
def print_test(test):
    print(json.dumps(test, indent=2, ensure_ascii=False))

#pass test_type as a list of types
def get_test_type(test_type, start=0, end=None, lower=0, upper=float('inf')):
    tests = [t for t in all_tests if t['type'] in test_type and lower <= t['char_count'] <= upper]
    return tests[start:end]

def get_random_tests(n=5, lower=0, upper=float('inf'), test_type=POSSIBLE_TYPES):
    filtered_tests = get_test_type(test_type=test_type, lower=lower, upper=upper) #[t for t in all_tests if lower <= t['char_count'] <= upper]
    sample_size = min(n, len(filtered_tests)) #prevent error
    return random.sample(filtered_tests, sample_size)

In [ ]:
tests = get_random_tests(upper=400, test_type=['math']) #get_test_type('math', end=10, lower=0, upper=500)
pprint(tests)

[{'char_count': 493,
  'expected': '36',
  'id': 'math_3_19_619',
  'prompt': 'Point $D$ lies on side $\\overline{BC}$ of $\\triangle ABC$ so '
            'that $\\overline{AD}$ bisects $\\angle BAC.$ The perpendicular '
            'bisector of $\\overline{AD}$ intersects the bisectors of $\\angle '
            'ABC$ and $\\angle ACB$ in points $E$ and $F,$ respectively. Given '
            'that $AB=4,BC=5,$ and $CA=6,$ the area of $\\triangle AEF$ can be '
            'written as $\\tfrac{m\\sqrt{n}}p,$ where $m$ and $p$ are '
            'relatively prime positive integers, and $n$ is a positive integer '
            'not divisible by the square of any prime. Find $m+n+p$ .',
  'type': 'math'},
 {'char_count': 475,
  'expected': '79',
  'id': 'math_3_20_620',
  'prompt': 'A hotel packed breakfast for each of three guests. Each breakfast '
            'should have consisted of three types of rolls, one each of nut, '
            'cheese, and fruit rolls. The preparer wrapped each o

In [33]:
#simple hello world call to kick off the commits
direct_call(prompt="how do I find the derivative of y=x^2 using python?")

OK: True HTTP: 200
MODEL SAYS: You can use the `sympy` library in Python to find the derivative of $ y = x^2 $. Here's the code:

```python
import sympy as sp

x = sp.symbols('x')
y = x**2
derivative = sp.diff(y, x)
print(derivative)
```

The output will be: `2*x`


In [ ]:
def interactive_chat():
    messages = ['<Start of message history>']
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Exiting chat.")
            break
        response = call_model_chat_completions(prompt=f"Old messages{messages}, CURRENT USER INPUT:{user_input} <--- ANSWER THIS QUESTION", temperature=0.7)
        messages.append(f"previous user input: {user_input}, previous system response: {response['text']}")
        if response["ok"]:
            print("Model:", response["text"].strip())
        else:
            print("Error:", response["error"])
        print(messages)
interactive_chat()

Model: 5
['<Start of message history>', 'previous user input: what is the square root of 25, previous system response: 5']
Model: 15
['<Start of message history>', 'previous user input: what is the square root of 25, previous system response: 5', "previous user input: what is the previous question's answer, plus 10, previous system response: 15"]
Model: 150
['<Start of message history>', 'previous user input: what is the square root of 25, previous system response: 5', "previous user input: what is the previous question's answer, plus 10, previous system response: 15", "previous user input: what is the previous question's answer, times 10, previous system response: 150"]
Model: 1
['<Start of message history>', 'previous user input: what is the square root of 25, previous system response: 5', "previous user input: what is the previous question's answer, plus 10, previous system response: 15", "previous user input: what is the previous question's answer, times 10, previous system respons

In [25]:
""" def execute_tests():
    rows = []
    for t in tests:
        r = call_model_chat_completions(
            prompt,
            system=system,
            model=model,
            temperature=0.3,
            max_tokens=128
        ) """

' def execute_tests():\n    rows = []\n    for t in tests:\n        r = call_model_chat_completions(\n            prompt,\n            system=system,\n            model=model,\n            temperature=0.3,\n            max_tokens=128\n        ) '

In [26]:
def self_evaluate(question, prediction, expected_answer, model=MODEL):
    """
    Use the model itself as a strict grader.
    Returns True if the model says the prediction matches the expected answer; else False.
    Falls back to a simple normalized string compare if the model's reply is malformed.
    """
    import re

    system = "You are a strict grader. Reply with exactly True or False. No punctuation. No explanation."
    prompt = f"""You are grading a question-answer pair.

Return exactly True if the PREDICTION would be accepted as correct for the EXPECTED_ANSWER.
Otherwise, return False.

QUESTION:
{question}

PREDICTION:
{prediction}

EXPECTED_ANSWER:
{expected_answer}

Answer with exactly: True or False
"""

    r = call_model_chat_completions(
        prompt,
        system=system,
        model=model,
        temperature=0.3,
    )

    reply = (r.get("text") or "").strip().lower()
    if reply.startswith("true"):
        return True
    if reply.startswith("false"):
        return False

    # No Fallback yet


In [55]:
def self_evaluate_tests(tests, model=MODEL, grader_model=None, sleep_sec=0.2, verbose=True):
    """
    Run the tests by querying the model for each prompt, then use LLM-as-a-judge
    (self_evaluate) to determine correctness.

    Args:
        tests: list of dicts with keys: id, prompt, expected (and optionally type)
        model: model used to generate predictions
        grader_model: model used to judge correctness (defaults to `model` if None)
        sleep_sec: small delay between calls to be polite to the API
        verbose: if True, print a summary line per test

    Returns:
        rows: list of dicts with fields:
              id, expected, got, correct, status, error
    """
    import time

    judge_model = grader_model or model
    rows = []
    count = 0
    for t in tests:
        count += 1
        # 1) Get model prediction
        #print('prompt:', t['prompt'])
        print_test(t)
        r = call_model_chat_completions(
            f"{t['prompt']}",
            system="Give a short answer to each prompt, don't explain.",
            model=model,
            temperature=0.3,
            max_tokens=128
        )
        got = (r.get("text") or "").strip()
        print(count, got)
        # 2) LLM-as-a-judge: strict True/False
        """ is_correct = self_evaluate(
            question=t["prompt"],
            prediction=got,
            expected_answer=t["expected"],
            model=judge_model,
        )

        row = {
            "id": t.get("id", "<unnamed>"),
            "expected": t["expected"],
            "got": got,
            "correct": bool(is_correct),
            "status": r.get("status"),
            "error": r.get("error"),
        }
        
        rows.append(row)
        print(json.dumps(row, indent=2, ensure_ascii=False))
        if verbose:
            mark = "✅" if is_correct else "❌"
            print(f"{mark} {row['id']}: expected={row['expected']!r}, got={row['got']!r} (HTTP {row['status']})")
            if row["error"]:
                print("   error:", row["error"]) """

        if sleep_sec:
            time.sleep(sleep_sec)

    return rows

# Example:


In [65]:
results_llm_judge = self_evaluate_tests(get_random_tests(n=3, upper=300), verbose=True, model=MODEL, grader_model=MODEL)


{
  "id": "math_3_249_849",
  "type": "math",
  "prompt": "There are 25 roses in a garden. There are 40 tulips. There are 35 daisies. What percentage of flowers are not roses?",
  "expected": "There are 25+40+35=<<25+40+35=100>>100 flowers total.\nThere are 40+35=<<40+35=75>>75 flowers that are not roses.\nTherefore, (75/100)*100=<<(75/100)*100=75>>75% of the flowers are not roses.\n#### 75",
  "char_count": 116
}
1 75%
{
  "id": "common_sense_1_203_303",
  "type": "common_sense",
  "prompt": "What American country music singer-songwriter, born in May of 1942, sang a duet with her ex-husband the same year that he released the song \"The Battle?\"",
  "expected": "Tammy Wynette",
  "char_count": 153
}
2 Dolly Parton
{
  "id": "math_3_100_700",
  "type": "math",
  "prompt": "Sally and Bob have made plans to go on a trip at the end of the year. They both decide to work as babysitters and save half of what they've earned for their trip. If Sally makes $6 per day and Bob makes $4 per day, h